In [83]:
# importing necessary libraries
import numpy as np
import astropy.units as u

In [191]:
def Read(filename):
    
    """
    This function will open and read any given data file according to this format,
    
        # Row 1: Time (Myr)  
        # Row 2: Total number of particles  
        # Row 3: Units for the following columns  
        # Row 4: Column headers  
        # Remaining rows contain particle data:  
        #   Col 1: Type (1=DM, 2=Disk, 3=Bulge)  
        #   Col 2: Mass (10^10 M⊙)  
        #   Col 3-5: Position (x, y, z) in kpc (MW center)  
        #   Col 6-8: Velocity (vx, vy, vz) in km/s (MW frame)  
        # Columns are tab-delimited  

    Inputs: filename (string) Name of the file in which the data is stored in the correct format.

    Output : time is the time in Myr
             total_particles is the total number of particles
             data is the full data array
    """
    file = open(filename, 'r')
    line1 = file.readline()
    label, value = line1.split()
    time = float(value)*u.Myr

    line2 = file.readline()
    label, value = line2.split()
    total_particles = float(value)
    file.close()

    # Skips the first three lines and starts storing from the 4th line
    data = np.genfromtxt(filename,dtype=None,names=True,skip_header=3)
    

    return (time, total_particles, data)

In [193]:
# Testing
time, total_particles, data = Read("MW_000.txt")

In [194]:
data['vx'][2] # This gives out the velocity in the x direction of the 2nd particle as a whole.

11.3833

In [197]:
# units of Myr (equivalent to SnapNumber x 10/0.7)
time # Should be 0

<Quantity 0. Myr>

In [199]:
index = np.where(data['type'] == 2)    # This is a disk type particle
x = data[index] # Gives an array whose particle type is a disk
x[100]['x'] # 100th disk particles' x distance in kpc)

14.87

In [203]:
data[-1]['vz'] #should be 16.7124

16.7124

In [273]:

def ParticleInfo(filename, particle_type, particle_number):
    """
    This function will compute the magnitude of the distance, magnitude of the velocity and mass of any given particle of 
    any given type (Disk, Halo, etc.)

    Inputs: filename (string) Name of the file in which the data is stored in the correct format.
            particle_type is the type of the particle, mapped to a number as follows, Type 1 = Dark Matter, 
            Type 2 = Disk Stars, Type 3 = Bulge Stars
            particle_number is the number of the particle

    Output : d (astropy units kpc)  Magnitude of the distance
             v (astropy units km/s) Magnitude of the velocity
             m (astropy units M_sun) Mass
    """
    
    time, total_particles, data = Read(filename)
    
    index = np.where(data['type'] == particle_type)
    data1 = data[index]


    # distance
    x = float(data1[particle_number]['x']) * u.kpc
    y = float(data1[particle_number]['y']) * u.kpc
    z = float(data1[particle_number]['z']) * u.kpc

    # velocity
    vx = float(data1[particle_number]['vx']) * u.km/u.s
    vy = float(data1[particle_number]['vy']) * u.km/u.s
    vz = float(data1[particle_number]['vz']) * u.km/u.s

    m = data['m'][index][particle_number] * 10**10* u.M_sun
   

    d = np.around((np.sqrt(x**2 + y**2 + z**2)), 3)
    v = np.around((np.sqrt(vx**2 + vy**2 + vz**2)), 3)

    d_3D = d.to(u.lyr)

    return (d, v, m, d_3D)

    

    

In [275]:
d,v, m, d_3D = ParticleInfo("MW_000.txt", 2, 99)
d # The 3D Distance in kpc

<Quantity 4.245 kpc>

In [277]:
v # The 3D Velocity in km/s

<Quantity 312.135 km / s>

In [281]:
m # The mass of the 100th disk particle in the milky way

<Quantity 1000000. solMass>

In [257]:
d_3D # The 3D Distance in light years

<Quantity 13845.33823408 lyr>

In [259]:

particle_type = 2
particle_num = 100
d, v, m, d_3D = ParticleInfo("MW_000.txt", particle_type, particle_num - 1)  # Disk stars are type 2
print(f"The properties of the {particle_num}th particle of the {particle_type} type")
print(f"Distance: {d}")
print(f"Velocity: {v}")
print(f"Mass: {m}")
print(f"3D Distance: {d_3D}")

The properties of the 100th particle of the 2 type
Distance: 4.245 kpc
Velocity: 312.135 km / s
Mass: 1000000.0 solMass
3D Distance: 13845.338234075754 lyr
